# Project 5: Optimizing Evacuation Routes using Real-Time Traffic Information

Song May, Michael Daugherty, Kelly Slatery | US-DSI-10 | 02.21.2020

## Problem Statement

## Executive Summary

## Data Dictionary

## Contents

# Data Collection

In [1]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import GetOldTweets3 as got
import time
import requests

In [2]:
# Credits (using GetOldTweets3 package): Martin Beck, 01.12.2020
# https://towardsdatascience.com/how-to-scrape-tweets-from-twitter-59287e20f0f1

# Set up variables for first Twitter pull
username = 'TxDOT'
since_date = '2016-01-01'
until_date = '2020-02-10'
max_tweets = 18000

# Create first query object
tweetCriteria = got.manager.TweetCriteria().setUsername(username).setSince(since_date).setUntil(until_date).setMaxTweets(max_tweets)

# Greate first list of all tweets
tweets = got.manager.TweetManager.getTweets(tweetCriteria)

# Create filtered list of tweet data
user_tweets = [[tweet.date, tweet.username, tweet.text, tweet.hashtags] for tweet in tweets]

# Transform list into the base tweet collection dataframe
all_tweets = pd.DataFrame(user_tweets)

An error occured during an HTTP request: HTTP Error 429: Too Many Requests
Try to open in browser: https://twitter.com/search?q=%20from%3Atxdot%20since%3A2016-01-01%20until%3A2020-02-10&src=typd


SystemExit: 

/Users/kelly/anaconda3/envs/dsi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Look at first 5 tweets from 'TxDOT'
pd.DataFrame(user_tweets).head()

In [ ]:
# Look at last 5 tweets from 'TxDOT'
pd.DataFrame(user_tweets).tail()

In [ ]:
# Define list of Twitter usernames to scrape, exclusing first Twitter user ('TxDOT')
# From: https://www.txdot.gov/driver/weather/txdot-twitter-feeds.html
twitter_users = ['TxDOTAbilene', 'TxDOTAmarillo', 'TxDOTAtlanta', 'TxDOTAustin', 
                 'TxDOTBeaumont', 'TxDOTBWD', 'TxDOTBryan', 'TxDOTChildress', 'TxDOT_CRP', 
                 'TxDOTDallas', 'TxDOTELP', 'TxDOTFortWorth', 'GalvestonFerry', 'TxDOTHouston', 
                 'HoustonTranstar', 'I35Travel', 'TxDOTLaredo', 'TxDOTLubbock', 'TxDOTLufkin', 
                 'TxDOTOdessa', 'TxDOTParis', 'TxDOTPharr', 'PortA_Ferry', 'TxDOTSanAngelo', 
                 'TxDOTSanAntonio', 'TexasHighways', 'TxDOTTyler', 'TxDOTWacoPIO', 'TXDOTWF', 
                 'TxDOTYoakum', 'ImproveMopac', 'ManorExpressway', 'DFWConnector', 'DriveMidtown', 
                 'Drive360South', 'LBJexpress', 'NTExpress', 'my290Houston']

In [ ]:
len(twitter_users)

In [ ]:
# Define a function to pull remaining users' tweets from the same time period
def update_tweets(base_df, user):
    
    # Set up variables for first Twitter pull
    username = user
    since_date = '2016-01-01'
    until_date = '2020-02-10'
    max_tweets = 18000

    # Create first query object
    tweetCriteria = got.manager.TweetCriteria().setUsername(username).setSince(since_date).setUntil(until_date).setMaxTweets(max_tweets)

    # Greate first list of all tweets
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)

    # Create filtered list of tweet data
    user_tweets = [[tweet.date, tweet.username, tweet.text, tweet.hashtags] for tweet in tweets]

    # Transform list into the base tweet collection dataframe
    tweets_df = pd.DataFrame(user_tweets)
    
    # Concatenate new tweets with old tweets
    updated = pd.concat([base_df, tweets_df], axis=0, ignore_index=True, sort=True)
    
    # Export the new dataframe individually
    tweets_df.to_csv(f'./data/data_by_user/{user}_tweets', index=False)
    
    # Print progress update
    print(f'Done scraping. We added {len(user_tweets)} new tweets. Current df shape: {updated.shape}')
    print()
    
    # Return updated dataframe of all tweets
    return updated

In [ ]:
# Get tweets from above 38 twitters users and add to the all_tweets dataframe

# Set up count
i = 0

# Loop through all twitter users to scrape and add to the all_tweets dataframe, according to Twitter API limits
for user in twitter_users:

    # Count twitters as they are scraped and get the initial time
    i += 1
    t0 = time.time()
    
    # Print progress update
    print(f'Twitter #{i} now scraping...')
    
    # Get the tweets and update the dataframe
    all_tweets = update_tweets(all_tweets, user)
    
    # Wait the the remainder of 15 minutes
    print(f'Time elapsed: {time.time() - t0}')
    time.sleep((15*60) - (time.time() - t0))
    

In [ ]:
# Look at first 5 tweets
all_tweets.head()

In [ ]:
# Look at last 5 tweets
all_tweets.tail()

# Export data

In [ ]:
all_tweets.to_csv('./data/DOT_tweets.csv', index=False)